## Краткий анализ предметной области

### Описание
Анализируется рынок золота — ключевой сегмент мирового финансового рынка. Стоимость золота зависит от макроэкономических показателей, геополитических событий, процентных ставок и спроса/предложения.

### Бизнес-задача
Провести анализ исторических данных о ценах на золото для выявления тенденций и факторов, влияющих на динамику стоимости.

### Основные проблемы
- Высокая волатильность цен
- Многофакторность влияния
- Реакция на мировые события

### Описание данных
**Источник:** Kaggle  
**Объём:** 18,000+ записей  
**Структура:** 7 столбцов (дата, цена, открытие, максимум, минимум, изменение %, валюта)

Данный датасет предоставляет широкие возможности для анализа динамики цен на золото, выявления закономерностей и построения аналитических моделей, что делает его ценным инструментом для финансовых аналитиков и инвесторов.

**Импорт библиотек и установка зависимостей**

In [4]:
# !pip install pandas numpy  # Установить необходимые библиотеки, если не установлены

import pandas as pd
import numpy as np

**Загрузка данных**

In [5]:
# Загрузка данных из файла CSV
# Замените путь на актуальный для вашего компьютера
file_path = 'gold.csv'
df = pd.read_csv(file_path)

**Описание темы и бизнес-задачи**

Тема: Анализ исторических данных о ценах на золото
Бизнес-задача:
Провести комплексный анализ динамики рынка золота, выявить ключевые тенденции, закономерности и факторы, влияющие на изменение стоимости золота. Сделать выводы, которые могут быть полезны для инвесторов и аналитиков.

**Изучение данных**

*Первичный обзор данных*

In [6]:
# Просмотр первых 5 строк
df.head()

,Date,Open,High,Low,Close,Volume,Currency
0,2000-01-04,289.5,289.5,280.0,283.7,21621,USD
1,2000-01-05,283.7,285.0,281.0,282.1,25448,USD
2,2000-01-06,281.6,282.8,280.2,282.4,19055,USD
3,2000-01-07,282.5,284.5,282.0,282.9,11266,USD
4,2000-01-10,282.4,283.9,281.8,282.7,30603,USD


In [7]:
# Просмотр последних 5 строк
df.tail()

,Date,Open,High,Low,Close,Volume,Currency
5698,2022-08-29,1748.4,1757.90,1731.40,1749.7,156220,USD
5699,2022-08-30,1749.8,1752.80,1732.90,1736.3,130664,USD
5700,2022-08-31,1735.5,1738.00,1720.60,1726.2,176731,USD
5701,2022-09-01,1723.0,1723.00,1699.10,1709.3,198618,USD
5702,2022-09-02,1707.9,1729.45,1707.05,1723.0,0,USD


In [8]:
# Случайная выборка 5 строк
df.sample(5, random_state=42)

,Date,Open,High,Low,Close,Volume,Currency
1436,2005-10-04,469.0,472.7,467.4,469.3,43497,USD
748,2003-01-03,346.0,353.5,343.6,351.6,37931,USD
3463,2013-10-11,1286.4,1294.8,1259.6,1268.2,186264,USD
1448,2005-10-20,465.8,467.4,462.0,463.2,59338,USD
4351,2017-04-24,1280.0,1280.0,1266.0,1277.5,302842,USD


In [9]:
# Атрибуты датафрейма
print('Форма датафрейма:', df.shape)
print('Названия столбцов:', list(df.columns))

Форма датафрейма: (5703, 7)
Названия столбцов: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Currency']


In [10]:
# Информация о датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5703 entries, 0 to 5702
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      5703 non-null   object 
 1   Open      5703 non-null   float64
 2   High      5703 non-null   float64
 3   Low       5703 non-null   float64
 4   Close     5703 non-null   float64
 5   Volume    5703 non-null   int64  
 6   Currency  5703 non-null   object 
dtypes: float64(4), int64(1), object(2)
memory usage: 312.0+ KB


In [11]:
# Использование памяти
df.memory_usage(deep=True)

,0
Index,132
Date,382101
Open,45624
High,45624
Low,45624
Close,45624
Volume,45624
Currency,342180


**Вывод:**

На данном этапе ознакомился с основными характеристиками датасета: размер, названия столбцов, типы данных, примерные значения.

*Описательная статистика и выявление проблем*

In [12]:
# Описательная статистика по числовым столбцам
df.describe()

,Open,High,Low,Close,Volume
count,5703.000000,5703.000000,5703.000000,5703.000000,5703.000000
mean,1040.382816,1048.339181,1031.863169,1040.298282,139141.669297
std,518.733377,522.353946,514.455903,518.524020,102537.449058
min,256.600000,259.400000,255.100000,256.600000,0.000000
25%,459.850000,463.900000,457.450000,460.500000,52938.500000
50%,1188.800000,1198.000000,1179.700000,1188.700000,126006.000000
75%,1381.400000,1392.750000,1368.100000,1383.050000,193109.000000
max,2076.400000,2089.200000,2049.000000,2069.400000,816531.000000


In [13]:
# Описательная статистика по всем столбцам
df.describe(include='all')

,Date,Open,High,Low,Close,Volume,Currency
count,5703,5703.000000,5703.000000,5703.000000,5703.000000,5703.000000,5703
unique,5703,NaN,NaN,NaN,NaN,NaN,1
top,2022-09-02,NaN,NaN,NaN,NaN,NaN,USD
freq,1,NaN,NaN,NaN,NaN,NaN,5703
mean,NaN,1040.382816,1048.339181,1031.863169,1040.298282,139141.669297,NaN
std,NaN,518.733377,522.353946,514.455903,518.524020,102537.449058,NaN
min,NaN,256.600000,259.400000,255.100000,256.600000,0.000000,NaN
25%,NaN,459.850000,463.900000,457.450000,460.500000,52938.500000,NaN
50%,NaN,1188.800000,1198.000000,1179.700000,1188.700000,126006.000000,NaN
75%,NaN,1381.400000,1392.750000,1368.100000,1383.050000,193109.000000,NaN


In [14]:
# Уникальные значения для категориальных столбцов
for col in df.select_dtypes(include='object').columns:
    print(f'{col}:', df[col].unique()[:10])

Date: ['2000-01-04' '2000-01-05' '2000-01-06' '2000-01-07' '2000-01-10'
 '2000-01-11' '2000-01-12' '2000-01-13' '2000-01-14' '2000-01-18']
Currency: ['USD']


**Вывод:**

Проведен анализ распределения данных, выявлены возможные проблемы: пропуски, выбросы, некорректные значения, несоответствие типов данных.

*Приведение наименований столбцов и текста к змеиному регистру*

In [15]:
# Приведение названий столбцов к snake_case и нижнему регистру
df.columns = [col.strip().replace(' ', '_').replace('-', '_').lower() for col in df.columns]
print('Новые названия столбцов:', list(df.columns))

# Приведение всех текстовых значений к нижнему регистру
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.lower()

Новые названия столбцов: ['date', 'open', 'high', 'low', 'close', 'volume', 'currency']


**Вывод:**

Названия столбцов и текстовые значения приведены к единому стилю, что облегчает дальнейшую обработку данных.

*Проверка и обработка пропусков*

In [16]:
# Проверка пропусков
missing = df.isnull().sum()
print(missing[missing > 0])

Series([], dtype: int64)


In [17]:
# Обработка пропусков
threshold = 0.05 * len(df)
for col in df.columns:
    if df[col].isnull().sum() > 0:
        if df[col].isnull().sum() <= threshold:
            # Если пропусков <5%, удаляем строки с пропусками
            df = df[df[col].notnull()]
            print(f'Столбец {col}: строки с пропусками удалены (менее 5% от общего числа)')
        elif df[col].dtype in ['float64', 'int64']:
            # Если числовой столбец — заполняем средним значением
            mean_value = df[col].mean()
            df[col] = df[col].fillna(mean_value)
            print(f'Столбец {col}: пропуски заполнены средним значением ({mean_value})')
        else:
            # Для категориальных — заполняем модой
            mode_value = df[col].mode()[0]
            df[col] = df[col].fillna(mode_value)
            print(f'Столбец {col}: пропуски заполнены модой ({mode_value})')

**Вывод:**

Проведена обработка пропусков: строки с небольшим количеством пропусков удалены, остальные заполнены средними значениями или модой в зависимости от типа данных.

*Изменение типов данных*

In [18]:
print('\nИзменение типов данных, если необходимо:')

# Преобразование столбца с датой в datetime
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])
    print('Столбец date: преобразован в datetime')

# Преобразование столбца с валютой в строковый тип
if 'currency' in df.columns:
    df['currency'] = df['currency'].astype(str)
    print('Столбец currency: преобразован в строковый тип')

# Обработка остальных столбцов
for col in df.columns:
    if df[col].dtype == 'object' and col not in ['date', 'currency']:
        try:
            # Пробуем преобразовать к float
            df[col] = df[col].astype(float)
            print(f'Столбец {col}: преобразован в float')
        except:
            # Оставляем как есть (категориальный)
            pass
# Объяснения: преобразуем к числовым или датам для корректной работы с данными и анализа


Изменение типов данных, если необходимо:
Столбец date: преобразован в datetime
Столбец currency: преобразован в строковый тип


**Вывод:**

Типы данных приведены к корректным для дальнейшего анализа: даты — к datetime, числовые — к float.

*Поиск и удаление дубликатов*

In [19]:
# Поиск дубликатов
dups = df.duplicated()
print(f'Количество дубликатов: {dups.sum()}')
if dups.sum() > 0:
    df = df.drop_duplicates()
    print('Дубликаты удалены.')
else:
    print('Дубликаты отсутствуют.')

Количество дубликатов: 0
Дубликаты отсутствуют.


**Вывод:**

Проведена проверка на дубликаты, при необходимости дубликаты удалены.

**Промежуточные выводы**

In [20]:
print(f'Итоговая форма датафрейма: {df.shape}')
print('Типы данных после обработки:')
print(df.dtypes)
print('Количество пропусков после обработки:')
print(df.isnull().sum().sum())

Итоговая форма датафрейма: (5703, 7)
Типы данных после обработки:
date        datetime64[ns]
open               float64
high               float64
low                float64
close              float64
volume               int64
currency            object
dtype: object
Количество пропусков после обработки:
0


**Вывод:**

Данные успешно предобработаны и готовы к дальнейшему анализу. Подавляющее большинство дупликатов удалено. Обработаны все пропуски. Названия столбцов приведены к змеиному стилю.